In [1]:
## 환경 세팅 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

pd.options.display.float_format = '{:.0f}'.format

def get_font_family():
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인
    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null
        import matplotlib.font_manager as fm
        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

get_font_family()
font_family = get_font_family()
plt.rc('font', family=font_family)
plt.rc('axes', unicode_minus=False)
plt.style.use('ggplot')

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test.head()

,Unnamed: 0,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1
0,0,15521,2014-09-12,AA,182320,A,2020-10-01,1,JRR
1,1,17651,2014-09-19,BB,82434,D,2020-10-01,1,J
2,2,26388,2014-10-25,BB,182320,A,2020-10-01,1,JRR
3,3,81772,2015-05-02,BB,204438,A,2020-10-01,1,DJ
4,4,92258,2015-05-24,AA,220745,A,2020-10-01,1,SD


In [3]:
train = train.drop(columns=['Unnamed: 0'])
test = test.drop(columns=['Unnamed: 0'])

In [4]:
train = train.drop(columns=['USER_ID'])
test = test.drop(columns=['USER_ID'])

In [5]:
train = train.drop(columns=['JOIN_DATE'])
test = test.drop(columns=['JOIN_DATE'])

In [6]:
train = train.drop(columns=['STORE_ID'])
test = test.drop(columns=['STORE_ID'])

In [7]:
train

,D_TYPE,GOODS_TYPE,DATE,COUNT,AD1
0,AA,A,2020-01-01,1,GN
1,BB,A,2020-01-01,1,J
2,BB,A,2020-01-01,1,SC
3,BB,C,2020-01-01,1,MP
4,AA,C,2020-01-01,1,JRR
...,...,...,...,...,...
623574,BB,B,2020-09-30,1,GN
623575,AA,B,2020-09-30,1,MP
623576,AA,B,2020-09-30,1,BF
623577,BB,A,2020-09-30,2,JRR


In [8]:
train['DATE'] = pd.to_datetime(train['DATE'])
test['DATE'] = pd.to_datetime(test['DATE'])

In [9]:
# 2020년 공휴일 지정
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday

class Calendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year', month=1, day=1),
        Holiday('KR_New Year1', month=1, day=24),
        Holiday('KR_New Year2', month=1, day=25),
        Holiday('KR_New Year3', month=1, day=26),
        Holiday('3.1', month=3, day=1),
        Holiday('Buddha Day', month=4, day=30),
        Holiday('Memorial Day', month=6, day=6),
        Holiday('Liberation Day', month=8, day=15),
        Holiday('KR_Thanksgiving1', month=9, day=30),
        Holiday('KR_Thanksgiving2', month=10, day=1),
        Holiday('KR_Thanksgiving3', month=10, day=2),
        Holiday('National Foundation Day', month=10, day=3),
        Holiday('Hangul Day', month=10, day=9),
        Holiday('Christmas', month=12, day=25)
    ]
    
    
dr = pd.date_range(start='2020-01-01', end='2020-12-31')
cal = Calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
train['Holiday'] = train['DATE'].isin(holidays)
test['Holiday'] = test['DATE'].isin(holidays)

In [10]:
from sklearn.preprocessing import LabelEncoder

def Holiday_to_int(data):
    le = LabelEncoder()
    le.fit([False, True])
    data['Holiday'] = le.transform(data['Holiday'])
    return data

train = Holiday_to_int(train)
test = Holiday_to_int(test)
train.head()

,D_TYPE,GOODS_TYPE,DATE,COUNT,AD1,Holiday
0,AA,A,2020-01-01,1,GN,1
1,BB,A,2020-01-01,1,J,1
2,BB,A,2020-01-01,1,SC,1
3,BB,C,2020-01-01,1,MP,1
4,AA,C,2020-01-01,1,JRR,1


In [11]:
X = train
Y = train.COUNT

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
logreg = LogisticRegression(solver='lbfgs')

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [16]:
from sklearn.compose import make_column_transformer

In [17]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['D_TYPE', 'GOODS_TYPE', 'AD1', 'DATE']),
    remainder='passthrough')

In [18]:
X = column_trans.fit_transform(X)

In [19]:
train = train.drop(columns=['DATE'])
test = test.drop(columns=['DATE'])

In [20]:
from sklearn.pipeline import make_pipeline

In [21]:
pipe = make_pipeline(column_trans, logreg)

In [22]:
cross_val_score(logreg, X, Y, cv=5, scoring='accuracy').mean()

/Users/tk/miniforge3/envs/project/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/tk/miniforge3/envs/project/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tk/miniforge3/envs/project/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Incre

0.9991388420764679